In [1]:
!pip install apyori
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from apyori import apriori
from itertools import combinations
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=70b31dd6da42e8d5ac8feb493fb5178b523a9809ff9eb260b390e772773baff4
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [2]:
%cd 'drive/My Drive/Colab Notebooks/ML TOR'
!pwd

/content/drive/My Drive/Colab Notebooks/ML TOR
/content/drive/My Drive/Colab Notebooks/ML TOR


In [3]:
df = pd.read_csv( 'TorontoData.csv')


In [4]:
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-8816401.413,5434587.125,701,GO-20141756319,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Commercial,1430,100,Assault,2014,March,24,83,Monday,3,2014.0,March,24.0,83.0,Monday,1,Assault,D42,132,Malvern (132),-79.199081,43.800281,1
1,-8837251.743,5413356.630,901,GO-20143006885,2014/09/27 00:00:00+00,2014/09/29 00:00:00+00,Other,2120,200,B&E,2014,September,29,272,Monday,10,2014.0,September,27.0,270.0,Saturday,16,Break and Enter,D52,76,Bay Street Corridor (76),-79.386383,43.662472,2
2,-8862433.492,5422275.764,702,GO-20141756802,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Commercial,2120,200,B&E,2014,March,24,83,Monday,7,2014.0,March,24.0,83.0,Monday,6,Break and Enter,D23,1,West Humber-Clairville (1),-79.612595,43.720406,3
3,-8833103.756,5431886.975,703,GO-20141760570,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Apartment,2120,200,B&E,2014,March,24,83,Monday,23,2014.0,March,24.0,83.0,Monday,15,Break and Enter,D33,47,Don Valley Village (47),-79.349121,43.782772,4
4,-8845310.751,5413667.148,902,GO-20142004859,2014/05/03 00:00:00+00,2014/05/03 00:00:00+00,Commercial,1610,210,Robbery - Business,2014,May,3,123,Saturday,2,2014.0,May,3.0,123.0,Saturday,2,Robbery,D11,90,Junction Area (90),-79.458778,43.664490,5


###Data Preprocessing

In [5]:
sampleX_Y = df[['occurrencedayofweek', 'Neighbourhood', 'MCI']]
sampleX_Y.head()

,occurrencedayofweek,Neighbourhood,MCI
0,Monday,Malvern (132),Assault
1,Saturday,Bay Street Corridor (76),Break and Enter
2,Monday,West Humber-Clairville (1),Break and Enter
3,Monday,Don Valley Village (47),Break and Enter
4,Saturday,Junction Area (90),Robbery


In [6]:
records=[]
for i in range(0,len(sampleX_Y)):
  records.append([str(sampleX_Y.values[i,j])for j in range(0,3)])
len(records)

206435

###Apriori Algorithm

In [7]:
#asso_rules = apriori(records, min_support= 0.3)
asso_rules = apriori(records)
asso_results = list(asso_rules)
#print(len(asso_results))
asso_results

[RelationRecord(items=frozenset({'Assault'}), support=0.5397485891442827, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Assault'}), confidence=0.5397485891442827, lift=1.0)]),
 RelationRecord(items=frozenset({'Auto Theft'}), support=0.11325598856783006, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Auto Theft'}), confidence=0.11325598856783006, lift=1.0)]),
 RelationRecord(items=frozenset({'Break and Enter'}), support=0.2097609417007775, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Break and Enter'}), confidence=0.2097609417007775, lift=1.0)]),
 RelationRecord(items=frozenset({'Friday    '}), support=0.15204301596144065, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Friday    '}), confidence=0.15204301596144065, lift=1.0)]),
 RelationRecord(items=frozenset({'Monday    '}), support=0.13954029113280209, ordered_statistics=[OrderedStatistic(i

#Data Processing

In [8]:
# Columns for the models
col_list = ['occurrenceyear',	'occurrencemonth','occurrenceday','occurrencedayofyear','occurrencedayofweek','occurrencehour','MCI',	'Division',	'Hood_ID','premisetype']

# New dataframe from columns
df2 = df[col_list]
df2 = df2[df2['occurrenceyear'] > 2013]

#Factorize
crime_var = pd.factorize(df2['MCI'])
df2['MCI'] = crime_var[0]
definition_list_MCI = crime_var[1]

premise_var = pd.factorize(df2['premisetype'])
df2['premisetype'] = premise_var[0]
definition_list_premise = premise_var[1] 

year_var = pd.factorize(df2['occurrenceyear'])
df2['occurrenceyear'] = year_var[0]
definition_list_year = year_var[1] 

month_var = pd.factorize(df2['occurrencemonth'])
df2['occurrencemonth'] = month_var[0]
definition_list_month = month_var[1] 

day_var = pd.factorize(df2['occurrenceday'])
df2['occurenceday'] = day_var[0]
definition_list_day = day_var[1] 

dayweek_var = pd.factorize(df2['occurrencedayofweek'])
df2['occurrencedayofweek'] = dayweek_var[0]
definition_list_day = dayweek_var[1] 

division_var = pd.factorize(df2['Division'])
df2['Division'] = division_var[0]
definition_list_division = division_var[1] 

hood_var = pd.factorize(df2['Hood_ID'])
df2['Hood_ID'] = hood_var[0]
definition_list_hood = hood_var[1] 

hour_var = pd.factorize(df2['occurrencehour'])
df2['occurrencehour'] = hour_var[0]
definition_list_hour = hour_var[1] 

dayyear_var = pd.factorize(df2['occurrencedayofyear'])
df2['occurrencedayofyear'] = dayyear_var[0]
definition_list_dayyear = dayyear_var[1]




#Splitting the dataset into training and tests set

In [9]:
#set X and Y:
X = df2.drop(['MCI'],axis=1).values
y = df2['MCI'].values

#split the data into train and test sets for numeric encoded dataset:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

#need to OneHotEncode all the X variables for input into the classification model:
binary_encoder = OneHotEncoder(sparse=False,categories='auto')
encoded_X = binary_encoder.fit_transform(X)
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size = 0.3, random_state = 21)

#RandomForestClassifier

In [10]:
#Balanced Class Weight doesn't make a big difference for results:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) 
y_pred


array([1, 0, 4, ..., 4, 0, 2])

In [11]:
print("Accuracy of Random Forest: ",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI))

Accuracy of Random Forest:  0.6378017182599209
[[23917  2186   631    53   819]
 [ 5460  5057    53    50   263]
 [ 2964   330  1639    19   388]
 [ 1098   374    34    26   129]
 [ 2766   707   239    29  2100]]
                 precision    recall  f1-score   support

        Assault       0.66      0.87      0.75     27606
Break and Enter       0.58      0.46      0.52     10883
        Robbery       0.63      0.31      0.41      5340
     Theft Over       0.15      0.02      0.03      1661
     Auto Theft       0.57      0.36      0.44      5841

       accuracy                           0.64     51331
      macro avg       0.52      0.40      0.43     51331
   weighted avg       0.61      0.64      0.61     51331



#KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [13]:
y_pred = classifier.predict(X_test)
y_pred

array([1, 1, 4, ..., 4, 1, 2])

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print("Accuracy of KNN: ",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy of KNN:  0.4779178274337145
[[17853  4654  2055   724  2320]
 [ 5346  3221   876   365  1075]
 [ 2116   816  1812   129   467]
 [  852   390   143    87   189]
 [ 2506  1105   496   175  1559]]
              precision    recall  f1-score   support

           0       0.62      0.65      0.63     27606
           1       0.32      0.30      0.31     10883
           2       0.34      0.34      0.34      5340
           3       0.06      0.05      0.06      1661
           4       0.28      0.27      0.27      5841

    accuracy                           0.48     51331
   macro avg       0.32      0.32      0.32     51331
weighted avg       0.47      0.48      0.47     51331



#AdaBoostClassifier

In [15]:
from sklearn.ensemble import AdaBoostClassifier
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,learning_rate=0.001)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
print("AdaBoostClassifier: ",accuracy_score(y_test, y_pred))

AdaBoostClassifier:  0.5378036664004208
